# Stockland Project 

Visitation Predition based on weather data.

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Import data set 
### 1. Stockland Visitation dataset: 

In [2]:
df_1 = pd.read_csv("data/Stockland-Visitation-Data.csv", encoding=None )
df_1.head()
df_1.count()

Asset               8190
Date                8190
Count visitation    7050
dtype: int64

### 2. Weather dataset (collected from BOM.gov.au)

In [3]:
df_2 = pd.read_csv("data/weather.csv", encoding=None)
df_2.head()
df_2.count()

Asset              8190
Date               8190
Min temperature    8068
Max temperature    8126
Solar exposure     8186
Rainfall           7886
Raining period     5597
dtype: int64

### 3. Merging the Visitation dataset and Weather dataset

In [4]:
df_full = pd.merge(df_1, df_2, on = ['Asset','Date'], how = "inner")
df_full.head()


,Asset,Date,Count visitation,Min temperature,Max temperature,Solar exposure,Rainfall,Raining period
0,Baldivis,1/01/2021,11878.0,19.9,30.0,30.7,0.0,NaN
1,Baldivis,2/01/2021,7962.0,19.5,32.6,30.2,0.0,NaN
2,Baldivis,3/01/2021,12918.0,18.0,32.0,24.3,0.0,NaN
3,Baldivis,4/01/2021,12796.0,17.3,32.7,30.9,0.0,NaN
4,Baldivis,5/01/2021,13321.0,18.2,34.2,30.8,0.0,NaN


In [5]:
df_full["Solar exposure"].dtypes

dtype('float64')

In [6]:
df_full.count()

Asset               8190
Date                8190
Count visitation    7050
Min temperature     8068
Max temperature     8126
Solar exposure      8186
Rainfall            7886
Raining period      5597
dtype: int64

## Cleaning data
### Filling missing values

After merge the two dataset, we observe that there are missing values in the data provided, which probably causing from they way data is collected. To resolve this problem, we apply a quick cleaning that run and check over all the missing values, and then imputing them with approriate value so that we can pulling them all into the model

In [7]:
# check na and missing values
df_full.isna().sum()

Asset                  0
Date                   0
Count visitation    1140
Min temperature      122
Max temperature       64
Solar exposure         4
Rainfall             304
Raining period      2593
dtype: int64

#### Impute missing data for rainfall and rain period: 
For rainfall, if there is a missing value, we assume that the rainfall is 0 (no rain). 
By looking at the raining period columns in the dataset collected from BOM, we define the backward and forward fill as the approriate method to impute missing value of the weather data, including Raining Period, Temperatures and Solar Exposure.

In [9]:
# imputing rainfall missing data with 0
df_full['Rainfall ']=df_full['Rainfall '].fillna(0)
# we use backwards fill for raining period 
df_full['Raining period']= df_full['Raining period'].bfill()
# if rainfall in a day is 0, then the raining period is 0
for i in range(len(df_full)):
    if df_full['Rainfall '].loc[i]==0:
        df_full['Raining period'].loc[i]=0 

In [11]:
## use forward fill to impute missing values of min temperature, max temperature and solar exposure
df_full['Max temperature']=df_full['Max temperature'].ffill()
df_full['Min temperature']=df_full['Min temperature'].ffill()
df_full['Solar exposure']=df_full['Solar exposure'].ffill()

### Create dummies variable for Assets and month 
Assets and months are considered as nominal categorical variables, so that one-hot encoding will be used for creating dummies variable (taking values of 1 or 0 to stand for true and false)

In [12]:
#create dummies variables for Location (Asset)
df_full = pd.get_dummies(df_full,columns=['Asset'])

In [13]:
## Extract month variable
from datetime import datetime 
for i in range(len(df_full)):   
    df_full['Date'].loc[i]= datetime.strptime(df_full['Date'].loc[i],'%d/%m/%Y')

df_full['Date'].loc[1].month
df_full['Month']= ""
for i in range(len(df_full)):
    df_full['Month'].loc[i]=df_full['Date'].loc[i].month

C:\Users\hungn\AppData\Local\Temp\ipykernel_22860\2109046020.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['Date'].loc[i]= datetime.strptime(df_full['Date'].loc[i],'%d/%m/%Y')
C:\Users\hungn\AppData\Local\Temp\ipykernel_22860\2109046020.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['Month'].loc[i]=df_full['Date'].loc[i].month


In [14]:
## convert into categorical month
df_full['Month']= df_full['Month'].astype(str)
df_full = pd.get_dummies(df_full,columns=['Month'])

In [15]:
# remove rows containing missing data in visitation
df_clean = df_full[df_full['Count visitation'].notna() ]

## Fiiting models 
### Prepare Model Input 

In [16]:
from sklearn.utils import shuffle
df_clean =shuffle(df_clean)

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

y_train = df_clean['Count visitation']
X_train = df_clean.drop(columns = ['Count visitation', 'Date'])

### 1. Fitting Multiple Linear Regression

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

lr = LinearRegression()
lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)

print(f'MSE train: {mean_squared_error(y_train, y_train_pred):.3f}')
print(f'R^2 train: {r2_score(y_train, y_train_pred):.3f}')

scores_lr = cross_val_score(estimator=lr, X=X_train, y=y_train, cv=10, n_jobs=1)

print(f'CV accuracy scores\n {scores_lr.reshape(-1,1)}')
print(f'CV accuracy: {np.mean(scores_lr):.3f} +/- {np.std(scores_lr):.3f}')

MSE train: 11002217.410
R^2 train: 0.794
CV accuracy scores
 [[0.73596457]
 [0.80490426]
 [0.80041495]
 [0.7803289 ]
 [0.8260849 ]
 [0.78273984]
 [0.77316103]
 [0.79391698]
 [0.81338186]
 [0.80788672]]
CV accuracy: 0.792 +/- 0.024


### 2.Decision Tree Regression

In [25]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=30)

tree.fit(X_train, y_train)

y_train_pred = tree.predict(X_train)

print(f'MSE train: {mean_squared_error(y_train, y_train_pred):.3f}')
print(f'R^2 train: {r2_score(y_train, y_train_pred):.3f}')

scores_lr = cross_val_score(estimator=tree, X=X_train, y=y_train, cv=10, n_jobs=1)

print(f'CV accuracy scores\n {scores_lr.reshape(-1,1)}')
print(f'CV accuracy: {np.mean(scores_lr):.3f} +/- {np.std(scores_lr):.3f}')


MSE train: 130427.133
R^2 train: 0.998
CV accuracy scores
 [[0.51779299]
 [0.60522393]
 [0.62500164]
 [0.65491861]
 [0.68639198]
 [0.58041772]
 [0.60336682]
 [0.66638931]
 [0.58792506]
 [0.61826939]]
CV accuracy: 0.615 +/- 0.046


### 3.Random Forrest Regressor

In [26]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()

forest.fit(X_train, y_train)

y_train_pred = forest.predict(X_train)

print(f'MSE train: {mean_squared_error(y_train, y_train_pred):.3f}')
print(f'R^2 train: {r2_score(y_train, y_train_pred):.3f}')

scores_lr = cross_val_score(estimator=forest, X=X_train, y=y_train, cv=10, n_jobs=1)

print(f'CV accuracy scores\n {scores_lr.reshape(-1,1)}')
print(f'CV accuracy: {np.mean(scores_lr):.3f} +/- {np.std(scores_lr):.3f}')

MSE train: 1558859.344
R^2 train: 0.971
CV accuracy scores
 [[0.74091284]
 [0.78921678]
 [0.8095057 ]
 [0.78889448]
 [0.81652837]
 [0.78887427]
 [0.7859544 ]
 [0.80175755]
 [0.78950755]
 [0.7905529 ]]
CV accuracy: 0.790 +/- 0.019


***Base on the above result, we will select the model with the highest accuracy score and $R^2$ value, which is the Random Forest Regressor.***

## **Hyper Parameter Tuning For Selected Model**

In [27]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in range(50,300,50)]
# Number of features to consider at every split
max_features = [int(x) for x in range(4,22)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
              'bootstrap': bootstrap}
print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 10 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 10, verbose=2, random_state=1, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
# Ouput the best params
rf_random.best_params_

{'n_estimators': [50, 100, 150, 200, 250], 'max_features': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'bootstrap': [True, False]}
Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'n_estimators': 250, 'max_features': 11, 'max_depth': 20, 'bootstrap': True}

In [34]:
# final = RandomForestRegressor(n_estimators= 200, max_features= 7, max_depth= 30, bootstrap= False)
# final.fit(X_train, y_train)

final = RandomForestRegressor(n_estimators= 250, max_features= 11, max_depth= 20, bootstrap= True)
final.fit(X_train, y_train)

y_train_pred = final.predict(X_train)

#print(f'MSE train: {mean_squared_error(y_train, y_train_pred):.3f}')
print(f'R^2 train: {r2_score(y_train, y_train_pred):.3f}')

R^2 train: 0.950


In [35]:
scores_lr = cross_val_score(estimator=final, X=X_train, y=y_train, cv=10, n_jobs=1)

# print(scores_v2)

print(f'CV accuracy scores\n {scores_lr.reshape(-1,1)}')
print(f'CV accuracy: {np.mean(scores_lr):.3f} +/- {np.std(scores_lr):.3f}')

CV accuracy scores
 [[0.7488358 ]
 [0.80083235]
 [0.81734921]
 [0.80034745]
 [0.83310278]
 [0.7994712 ]
 [0.79500923]
 [0.80739552]
 [0.80318788]
 [0.80388987]]
CV accuracy: 0.801 +/- 0.020
